In [30]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
df = pl.read_csv("HousePricePrediction.csv")

In [4]:
df.head()

status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
str,i64,i64,f64,str,str,i64,i64,str,i64
"""for_sale""",3,2,0.12,"""Adjuntas""","""Puerto Rico""",601,920,null,105000
"""for_sale""",4,2,0.08,"""Adjuntas""","""Puerto Rico""",601,1527,null,80000
"""for_sale""",2,1,0.15,"""Juana Diaz""","""Puerto Rico""",795,748,null,67000
"""for_sale""",4,2,0.1,"""Ponce""","""Puerto Rico""",731,1800,null,145000
"""for_sale""",6,2,0.05,"""Mayaguez""","""Puerto Rico""",680,null,null,65000


In [6]:
print(df.describe())

shape: (9, 11)
┌────────────┬───────────┬──────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ statistic  ┆ status    ┆ bed      ┆ bath     ┆ … ┆ zip_code  ┆ house_siz ┆ prev_sold ┆ price     │
│ ---        ┆ ---       ┆ ---      ┆ ---      ┆   ┆ ---       ┆ e         ┆ _date     ┆ ---       │
│ str        ┆ str       ┆ f64      ┆ f64      ┆   ┆ f64       ┆ ---       ┆ ---       ┆ f64       │
│            ┆           ┆          ┆          ┆   ┆           ┆ f64       ┆ str       ┆           │
╞════════════╪═══════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ count      ┆ 904966    ┆ 775126.0 ┆ 791082.0 ┆ … ┆ 904762.0  ┆ 612080.0  ┆ 445865    ┆ 904895.0  │
│ null_count ┆ 0         ┆ 129840.0 ┆ 113884.0 ┆ … ┆ 204.0     ┆ 292886.0  ┆ 459101    ┆ 71.0      │
│ mean       ┆ null      ┆ 3.33219  ┆ 2.484236 ┆ … ┆ 6519.4645 ┆ 2138.4366 ┆ null      ┆ 877438.15 │
│            ┆           ┆          ┆          ┆   ┆ 82        ┆ 67        ┆

In [8]:
df = df.drop_nulls()

In [26]:
# Get the list of columns with string dtype
string_columns = [col for col in df.columns if df[col].dtype == pl.Utf8]

# Drop the original string columns
df = df.drop(string_columns)

In [27]:
df.head()

bed,bath,acre_lot,zip_code,house_size,price
i64,i64,f64,i64,i64,i64
7,3,0.09,949,1192,110000
7,3,0.09,949,1192,110000
7,3,0.09,949,1192,110000
7,3,0.09,949,1192,110000
7,3,0.09,949,1192,110000


In [31]:
# Get the list of columns with string dtype
string_columns = [col for col in df.columns if df[col].dtype == pl.Utf8]

# Create dummy variables for string columns
for col in string_columns:
    unique_values = df[col].unique()
    for val in unique_values:
        df[val] = (df[col] == val).then_else(pl.lit(1), pl.lit(0))

# Drop the original string columns
df = df.drop(string_columns)

# Define the features (X) and target (y)
X = df.drop("price").to_numpy()
y = df["price"].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Print the intercept and coefficients
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# Predict on the test set
y_pred = model.predict(X_test)

# Compute the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Compute the R-squared score
score = r2_score(y_test, y_pred)
print("R-squared Score:", score)

Intercept: -433553.0769208628
Coefficients: [-7.27828880e+04  4.61638265e+05  9.23169093e+00 -4.04800945e+00
  1.10199889e+02]
Mean Squared Error: 1244305776778.698
R-squared Score: 0.3042027411827608
